In [ ]:
!pip install foolbox==2.3.0

In [ ]:
import foolbox
import numpy as np
import torch
import torchvision.models as models
from PIL import Image
import torchvision.transforms as transforms
from torch.autograd import Variable
from scipy.special import softmax
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import time 

In [ ]:
# preprocess original image 
def preprocess_orig_image(image):

    im_orig = Image.open(image)
    im = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor()])(im_orig)
    im = im.reshape(3,224,224)
    return im

# create foolbox model given the PyTorch model
def test_model(model):
    model = model.eval()
    preprocessing = dict(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], axis=-3)
    fmodel = foolbox.models.PyTorchModel(model, bounds=(0, 1), num_classes=1000, preprocessing=preprocessing)
    return fmodel


def get_labels(label_orig, label_pert):

    labels = open(os.path.join('synset_words.txt'), 'r').read().split('\n')

    str_label_orig = labels[np.int(label_orig)].split(',')[0]
    str_label_pert = labels[np.int(label_pert)].split(',')[0]
    
    return str_label_orig,str_label_pert


# comptute the cosine similarity between 2 original image and adversarial image
def calculate_cossim(orig,pert):

    original_image = orig.flatten()
    perturbed_image = pert.flatten()

    dot_product = np.dot(original_image, perturbed_image)
    norm_a = np.linalg.norm(original_image)
    norm_b = np.linalg.norm(perturbed_image)
    cossimi = ((dot_product / (norm_a * norm_b)))
    return cossimi


def mse(imageA, imageB):
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])

    # return the MSE, the lower the error, the more "similar"
    # the two images are
    return err

In [ ]:
model = models.vgg19(pretrained=True) 
fmodel = test_model(model)

images, labels = foolbox.utils.samples(dataset='imagenet',batchsize=20, data_format='channels_first',bounds=(0,1))
print(np.mean(fmodel.forward(images).argmax(axis=-1) == labels))

print ("Running Attack")
start_time = time.time()
attack = foolbox.v1.attacks.LBFGSAttack(fmodel, distance=foolbox.distances.MeanSquaredDistance)
adversarials = []
for i in range(len(images)):
  adversarials.append(attack(images[i], labels[i]))
end_time = time.time() - start_time

print ("Time taken: %.2f"%end_time)

# print (adversarials[0].perturbed)

im_advs_confidence = []
im_orig_confidence = []
cos_sim = []
mse_val = []

i = 0
for adv in adversarials:
  image_conf = np.amax(foolbox.utils.softmax(fmodel.forward_one(images[i])))
  im_orig_confidence.append(image_conf)
  advs_conf = np.amax(foolbox.utils.softmax(fmodel.forward_one(adv)))
  im_advs_confidence.append(advs_conf)
  orig = np.transpose(adv, (1, 2, 0))
  pert = np.transpose(images[i], (1, 2, 0))
  cos_sim.append(calculate_cossim(orig,pert))
  mse_val.append(mse(orig,pert))
  i+=1

print ("Average Cosine Sim: ",np.mean(np.asarray(cos_sim)))
print ("Average MSE: ",np.mean(np.asarray(mse_val)))
print ("Original Conf %.2f%%: " % (np.mean(np.asarray(im_orig_confidence))*100))
print ("Adversarial %.2f%%: " % (np.mean(np.asarray(im_advs_confidence))*100))


0.95
Running Attack


/usr/local/lib/python3.6/dist-packages/foolbox/v1/attacks/base.py:141: UserWarning: Not running the attack because the original input is already misclassified and the adversarial thus has a distance of 0.
  "Not running the attack because the original input"


Time taken: 884.85
Average Cosine Sim:  0.9999989
Average MSE:  2.011296134913201e-06
Original Conf 85.51%: 
Adversarial 88.32%: 
